In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

root = "../data/"

### 1. 处理泌乳数据

In [2]:
cols = ["牛号", "日产量", "日期", "日活动量", "日反刍量", "泌乳天数", "泌乳期"]
raw_lact = pd.read_excel(os.path.join(root, '泌乳数据 2021_10_9 16-41-26.xlsx'), usecols=cols)

d:\Program\Python3.10\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
raw_lact.shape
raw_lact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248866 entries, 0 to 248865
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   牛号      248866 non-null  int64         
 1   日产量     210557 non-null  float64       
 2   日期      248866 non-null  datetime64[ns]
 3   日活动量    246547 non-null  float64       
 4   日反刍量    246086 non-null  float64       
 5   泌乳天数    248866 non-null  int64         
 6   泌乳期     248866 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(3)
memory usage: 13.3 MB


In [4]:
raw_lact["日产量"].fillna(0, inplace=True)

In [5]:
raw_lact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248866 entries, 0 to 248865
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   牛号      248866 non-null  int64         
 1   日产量     248866 non-null  float64       
 2   日期      248866 non-null  datetime64[ns]
 3   日活动量    246547 non-null  float64       
 4   日反刍量    246086 non-null  float64       
 5   泌乳天数    248866 non-null  int64         
 6   泌乳期     248866 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(3)
memory usage: 13.3 MB


In [6]:
# 去重
raw_lact.drop_duplicates(inplace=True)

In [7]:
raw_lact.describe()

,牛号,日产量,日活动量,日反刍量,泌乳天数,泌乳期
count,248866.000000,248866.000000,246547.000000,246086.000000,248866.00000,248866.000000
mean,145209.595762,30.647680,489.705010,533.480283,191.74299,2.539865
std,64392.578868,16.639209,75.557965,88.760251,128.37222,1.421493
min,9.000000,0.000000,20.000000,1.000000,0.00000,1.000000
25%,160157.000000,22.552999,444.000000,496.000000,79.00000,1.000000
50%,170393.000000,33.875999,484.000000,544.000000,188.00000,2.000000
75%,180426.000000,42.349998,530.000000,587.000000,287.00000,3.000000
max,190580.000000,87.351997,1287.000000,1276.000000,1174.00000,6.000000


In [8]:
# 把反刍量小于 50 的数据去除

lact = raw_lact[raw_lact["日反刍量"] > 50]

In [9]:
lact = lact[lact["日期"] >= pd.to_datetime("2021-03-01")]

In [10]:
print(lact.shape)
print(raw_lact.shape)

(199349, 7)
(248866, 7)


### 2. 处理疾病数据

In [11]:
dis_cols = ["管理号", "疾病名称", "疾病分类", "发病日期", "治愈日期"]
raw_dis = pd.read_excel(os.path.join(root, "疾病记录(2021年10月09日).xlsx"), usecols=dis_cols)

In [12]:
raw_dis.shape
raw_dis.columns
raw_dis.head()

,管理号,疾病名称,疾病分类,发病日期,治愈日期
0,160122,乳房炎,乳房疾病,2021-03-01,2021-03-01
1,180299,乳房炎,乳房疾病,2021-03-01,2021-03-01
2,170473,子宫炎,生殖系统,2021-03-02,2021-03-20
3,170493,前胃迟缓,消化系统,2021-03-03,2021-03-06
4,160039,隐性乳房炎,自定义疾病,2021-03-07,2021-03-07


In [13]:
raw_dis["疾病分类"].value_counts()

乳房疾病      192
自定义疾病     112
生殖系统       63
消化系统       29
营养代谢系统     19
应激与过敏       5
泌尿系统        2
四肢疾病        2
呼吸系统        1
传染病         1
神经系统        1
Name: 疾病分类, dtype: int64

1. 把疾病发现到治愈周期内都标注为疾病状态
2. 把尚未治愈的情况，定义后面5天都是疾病状态
3. 把疾病发现时间之前10数据删除，治愈日期后10天数据删除

用170473 170493 测试

In [14]:
raw_dis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427 entries, 0 to 426
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   管理号     427 non-null    int64 
 1   疾病名称    427 non-null    object
 2   疾病分类    427 non-null    object
 3   发病日期    427 non-null    object
 4   治愈日期    427 non-null    object
dtypes: int64(1), object(4)
memory usage: 16.8+ KB


In [15]:
# 把发病日期，治愈日期转化为日期类型

raw_dis["发病日期"] = raw_dis["发病日期"].astype(np.datetime64)

In [16]:
# raw_dis['治愈日期']=raw_dis['治愈日期'].astype(np.datetime64)

In [17]:
from datetime import timedelta


def update_date(row):
    if row["治愈日期"] == "尚未治愈":
        row["治愈日期"] = row["发病日期"] + timedelta(days=5)
    return row

In [18]:
dis_update = raw_dis.apply(update_date, axis=1)

In [19]:
# dis_update['治愈日期'].unique()

In [20]:
dis_update["治愈日期"] = dis_update["治愈日期"].astype(np.datetime64)

In [21]:
dis_update.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427 entries, 0 to 426
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   管理号     427 non-null    int64         
 1   疾病名称    427 non-null    object        
 2   疾病分类    427 non-null    object        
 3   发病日期    427 non-null    datetime64[ns]
 4   治愈日期    427 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 16.8+ KB


### 3. 合并数据


In [22]:
lact.head()

,牛号,日产量,日期,日活动量,日反刍量,泌乳天数,泌乳期
59,6443,36.905998,2021-03-01,386.0,635.0,16,5
60,6443,40.021999,2021-03-02,393.0,654.0,17,5
61,6443,39.776001,2021-03-03,410.0,665.0,18,5
62,6443,41.172001,2021-03-04,382.0,564.0,19,5
63,6443,38.122002,2021-03-05,383.0,393.0,20,5


In [23]:
dis_update.head()

,管理号,疾病名称,疾病分类,发病日期,治愈日期
0,160122,乳房炎,乳房疾病,2021-03-01,2021-03-01
1,180299,乳房炎,乳房疾病,2021-03-01,2021-03-01
2,170473,子宫炎,生殖系统,2021-03-02,2021-03-20
3,170493,前胃迟缓,消化系统,2021-03-03,2021-03-06
4,160039,隐性乳房炎,自定义疾病,2021-03-07,2021-03-07


In [24]:
data = pd.merge(
    lact, dis_update, left_on=["牛号", "日期"], right_on=["管理号", "发病日期"], how="left"
)

In [25]:
data[data["管理号"].notnull()]

,牛号,日产量,日期,日活动量,日反刍量,泌乳天数,泌乳期,管理号,疾病名称,疾病分类,发病日期,治愈日期
784,4363,2.489000,2021-06-24,531.0,446.0,445,5,4363.0,乳房炎,乳房疾病,2021-06-24,2021-06-24
905,6280,1.671000,2021-03-15,620.0,533.0,444,4,6280.0,乳房炎,乳房疾病,2021-03-15,2021-03-15
997,6280,48.643002,2021-06-15,505.0,487.0,35,5,6280.0,乳房炎,乳房疾病,2021-06-15,2021-06-20
1721,7328,4.541000,2021-08-09,356.0,259.0,39,5,7328.0,乳房炎,乳房疾病,2021-08-09,2021-08-19
1732,7328,37.860001,2021-08-20,482.0,604.0,50,5,7328.0,乳房炎,乳房疾病,2021-08-20,2021-08-25
...,...,...,...,...,...,...,...,...,...,...,...,...
196594,190477,28.180000,2021-08-21,466.0,566.0,27,1,190477.0,子宫炎,生殖系统,2021-08-21,2021-08-21
196692,190480,10.860000,2021-08-18,596.0,371.0,11,1,190480.0,子宫炎,生殖系统,2021-08-18,2021-09-02
196767,190481,31.135000,2021-08-06,472.0,603.0,24,1,190481.0,子宫炎,生殖系统,2021-08-06,2021-08-17
197636,190500,37.860001,2021-09-09,497.0,471.0,15,1,190500.0,子宫炎,生殖系统,2021-09-09,2021-09-14


In [26]:
data_notnull_index = data[data["管理号"].notnull()].index

In [27]:
data_notnull_index

Int64Index([   784,    905,    997,   1721,   1732,   2648,   2733,   3083,
              3309,   3433,
            ...
            194844, 194919, 195741, 195747, 196337, 196594, 196692, 196767,
            197636, 197747],
           dtype='int64', length=404)

In [28]:
data["新增疾病类型"] = np.nan

####    把疾病发现到治愈周期内都标注为疾病状态

In [29]:
for index in data_notnull_index:
    row = data.iloc[index]
    start = row["发病日期"]
    end = row["治愈日期"]
    dif = pd.date_range(start, end)
    cow_index = row["牛号"]
    i = index
    for d in dif:
        # data[(data['牛号']==cow_index) & data['日期']=d]['新增疾病类型']=row['疾病分类']
        data["新增疾病类型"].iloc[i] = row["疾病分类"]
        i += 1

C:\Users\10069\AppData\Local\Temp\ipykernel_2436\3674830551.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["新增疾病类型"].iloc[i] = row["疾病分类"]


In [30]:
data[(data["牛号"] == 170493) & (data["新增疾病类型"].notnull())]
# 用 170473 170493 测试

,牛号,日产量,日期,日活动量,日反刍量,泌乳天数,泌乳期,管理号,疾病名称,疾病分类,发病日期,治愈日期,新增疾病类型
108729,170493,25.280001,2021-04-21,410.0,591.0,160,2,170493.0,蹄病,自定义疾病,2021-04-21,2021-05-18,自定义疾病
108730,170493,22.167999,2021-04-22,431.0,548.0,161,2,NaN,NaN,NaN,NaT,NaT,自定义疾病
108731,170493,27.410000,2021-04-23,410.0,510.0,162,2,NaN,NaN,NaN,NaT,NaT,自定义疾病
108732,170493,26.908001,2021-04-24,429.0,509.0,163,2,NaN,NaN,NaN,NaT,NaT,自定义疾病
108733,170493,21.902000,2021-04-25,458.0,423.0,164,2,NaN,NaN,NaN,NaT,NaT,自定义疾病
108734,170493,8.122000,2021-04-26,433.0,631.0,165,2,NaN,NaN,NaN,NaT,NaT,自定义疾病
108735,170493,17.764999,2021-04-27,431.0,542.0,166,2,NaN,NaN,NaN,NaT,NaT,自定义疾病
108736,170493,8.502000,2021-04-28,393.0,603.0,167,2,NaN,NaN,NaN,NaT,NaT,自定义疾病
108737,170493,17.593000,2021-04-29,398.0,510.0,168,2,NaN,NaN,NaN,NaT,NaT,自定义疾病
108738,170493,19.000000,2021-04-30,425.0,465.0,169,2,NaN,NaN,NaN,NaT,NaT,自定义疾病


In [31]:
data[data["新增疾病类型"].notnull()]

,牛号,日产量,日期,日活动量,日反刍量,泌乳天数,泌乳期,管理号,疾病名称,疾病分类,发病日期,治愈日期,新增疾病类型
784,4363,2.489000,2021-06-24,531.0,446.0,445,5,4363.0,乳房炎,乳房疾病,2021-06-24,2021-06-24,乳房疾病
905,6280,1.671000,2021-03-15,620.0,533.0,444,4,6280.0,乳房炎,乳房疾病,2021-03-15,2021-03-15,乳房疾病
997,6280,48.643002,2021-06-15,505.0,487.0,35,5,6280.0,乳房炎,乳房疾病,2021-06-15,2021-06-20,乳房疾病
998,6280,22.277000,2021-06-16,510.0,430.0,36,5,NaN,NaN,NaN,NaT,NaT,乳房疾病
999,6280,16.101999,2021-06-17,496.0,373.0,37,5,NaN,NaN,NaN,NaT,NaT,乳房疾病
...,...,...,...,...,...,...,...,...,...,...,...,...,...
197748,190502,39.389999,2021-09-13,539.0,563.0,24,1,NaN,NaN,NaN,NaT,NaT,生殖系统
197749,190502,34.597000,2021-09-14,536.0,657.0,25,1,NaN,NaN,NaN,NaT,NaT,生殖系统
197750,190502,35.306999,2021-09-15,513.0,638.0,26,1,NaN,NaN,NaN,NaT,NaT,生殖系统
197751,190502,40.227001,2021-09-16,502.0,659.0,27,1,NaN,NaN,NaN,NaT,NaT,生殖系统


In [32]:
data_bak = data.copy()  # data_bak :把疾病发现到治愈周期内都标注为疾病状态

#### 

In [33]:
# 统计要删除的行数
del_row = []
# index=999
for index in data_notnull_index:
    row = data.iloc[index]
    start = row["发病日期"]
    end = row["治愈日期"]
    dif = pd.date_range(start, end)
    # finger 日期之前的 10 天

    end_index = index + len(dif) - 1
    for i in range(1, 10):
        del_row.append(index - i)
        del_row.append(end_index + i)

In [34]:
len(del_row)

7272

In [35]:
print(data_bak.shape)

(199350, 13)


In [36]:
data.drop(del_row, inplace=True)

In [37]:
data.shape

(192218, 13)

In [38]:
# data[data['牛号']==170473][:30]

In [39]:
data.to_csv(os.path.join(root, "sample.csv"))